In [1]:
import os
import pickle
import json
import numpy as np
from keras.layers import Input,Dense,Conv2D,Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.models import Model
from sklearn.metrics import f1_score
import pickle
from keras.layers import BatchNormalization

/home/parmeet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/parmeet/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
with open('inception_results.pckl','rb') as inception_res:
    inception_val,true_labels,inception_test_val = pickle.load(inception_res)
with open('nasnet_results.pckl','rb') as nasnet_res:
    nasnet_val,true_labels,nasnet_test_val = pickle.load(nasnet_res)
with open('resnet_results.pckl','rb') as resnet_res:
    resnet_val,true_labels,resnet_test_val = pickle.load(resnet_res)
with open('vgg_results.pckl','rb') as vgg_res:
    vgg_val,true_labels,vgg_test_val = pickle.load(vgg_res)

In [3]:
number_of_models = 4
number_of_classes = 229

In [4]:
input_data = np.dstack((inception_val,nasnet_val,resnet_val,vgg_val))

In [5]:
input_dt = np.expand_dims(input_data,axis=3)

In [6]:
inp = Input(shape=(number_of_classes,number_of_models,1)) 

In [7]:
middle = Conv2D(16, (1, 2))(inp)
middle = Conv2D(32, (1, 3))(middle)
middle = BatchNormalization()(middle)
middle = Flatten()(middle)
middle = Dense(1024)(middle)
middle = BatchNormalization()(middle)
final = Dense(number_of_classes, activation='sigmoid')(middle)
model = Model(inputs= inp,outputs=final)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 229, 4, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 229, 3, 16)        48        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 229, 1, 32)        1568      
_________________________________________________________________
batch_normalization_1 (Batch (None, 229, 1, 32)        128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 7328)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              7504896   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
__________

In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
for train,test in kf.split(input_dt):
    model.compile(optimizer='adam',loss='binary_crossentropy')
    model.fit(input_dt[train],np.array(true_labels)[train],epochs=10)
    predicted = model.predict(input_dt[test])
    predicted[predicted>=0.3] = 1
    predicted[predicted<0.3] = 0
    print(f1_score(np.array(np.array(true_labels)[test]),predicted,average='micro'))

Epoch 1/10
6598/6598 [==============================] - 5s 701us/step - loss: 0.2670
Epoch 2/10
6598/6598 [==============================] - 3s 385us/step - loss: 0.0750
Epoch 3/10
6598/6598 [==============================] - 3s 388us/step - loss: 0.0722
Epoch 4/10
6598/6598 [==============================] - 3s 386us/step - loss: 0.0703
Epoch 5/10
6598/6598 [==============================] - 3s 392us/step - loss: 0.0694
Epoch 6/10
6598/6598 [==============================] - 3s 385us/step - loss: 0.0681
Epoch 7/10
6598/6598 [==============================] - 3s 384us/step - loss: 0.0674
Epoch 8/10
6598/6598 [==============================] - 3s 386us/step - loss: 0.0653
Epoch 9/10
6598/6598 [==============================] - 3s 385us/step - loss: 0.0650
Epoch 10/10
6598/6598 [==============================] - 3s 394us/step - loss: 0.0632
0.6005466322213486
Epoch 1/10
6598/6598 [==============================] - 3s 428us/step - loss: 0.0715
Epoch 2/10
6598/6598 [=======================

In [10]:
predicted = model.predict(input_dt)
predicted[predicted>=0.3] = 1
predicted[predicted<0.3] = 0
print(f1_score(np.array(true_labels),predicted,average='micro'))

0.6777304601587678


In [11]:
predicted = model.predict(np.expand_dims(np.dstack((inception_test_val,nasnet_test_val,resnet_test_val,vgg_test_val)),axis=3))
predicted[predicted>=0.3] = 1
predicted[predicted<0.3] = 0

In [12]:
outputs = []
for i in range(predicted.shape[0]):
        arr = np.where(predicted[i]==1)[0]
        outputs.append(" ".join(map(str, arr.tolist())))

In [13]:
import pandas as pd
df = pd.DataFrame()

In [14]:
with open("test_data_vgg19.pckl",'rb') as write_file:
    image_features,img_ids = pickle.load(write_file)

In [15]:
df['image_id'] = img_ids
df['label_id'] = outputs

In [16]:
df.to_csv("predictions.csv",index=False)